In [1]:
using Printf
using DelimitedFiles
using LinearAlgebra
#
const sc_size = 1
const print_to_file = true
const fname = "coordinates.xyz"
#
function read_poscar()
    fname="POSCAR"
    poscar = open(fname)
    lines = readlines(fname)
    lvec1 = parse.(Float64,split(lines[3]))'
    lvec2 = parse.(Float64,split(lines[4]))'
    lvec3 = parse.(Float64,split(lines[5]))'
    latt_vec = vcat(lvec1,lvec2,lvec3)
    types = split(lines[6])
    ntypes = size(types,1)
    natoms = parse.(Int,split(lines[7]))
    nat=0
    atom=0
    ityp=String[]
    for t = 1:ntypes
    for i = 1:natoms[t]
        atom += 1
        push!(ityp,types[t])
    end
    end
    nat = atom
    close(poscar)
    #
    (lines[8][1]=='C') ? cartesian = true : cartesian = false
    tau = zeros(0,3)
    for i = 1 : nat
        row = parse.(Float64,split(lines[8+i]))'
        tau = vcat(tau,row)
        if (!cartesian)
           tau[i,:] = latt_vec * tau[i,:]
        end
     end    
    return (ityp, latt_vec, tau)
end
#
#
function main()
(ityp, latt_vec, tau) = read_poscar()
nat = size(tau,1)
tau_sc = tau
tau_tmp = zeros(3)
#
for nx = -sc_size : sc_size
    for ny = -sc_size : sc_size
        for nz = -sc_size : sc_size
            #
            if (nx!=0)||(ny!=0)||(nz!=0)
             #
             for na = 1:nat
                 tau_tmp[:] = tau[na,:] + nx*latt_vec[1,:] + ny*latt_vec[2,:] + nz*latt_vec[3,:]
                 tau_sc = vcat(tau_sc, tau_tmp' )
                 push!(ityp, ityp[na])
             end
             #
            end
            #
        end                             
    end
end
#
function final_print(tofile)
  nat_sc = nat * (2*sc_size+1)^3
  if (!tofile)
    println(nat_sc)
    println("xyz")
    for na = 1: nat_sc
        @printf("%s %10.8f %10.8f %10.8f %6i \n", ityp[na], tau_sc[na,1], tau_sc[na,2], tau_sc[na,3], na) 
    end
  else         
    outfile = open(fname, "w")  
    println(outfile, nat_sc)
    println(outfile, "xyz")
    for na = 1: nat_sc
        @printf(outfile,"%s %10.8f %10.8f %10.8f %6i \n", ityp[na], tau_sc[na,1], tau_sc[na,2], tau_sc[na,3], na) 
    end
    close(outfile)
  end
end

final_print(print_to_file)

end


@time begin

    main()

end

  0.386465 seconds (652.51 k allocations: 37.644 MiB, 4.48% gc time, 97.74% compilation time)
